#### Import modules and define parameters


In [1]:
from pathlib import Path

import pandas as pd

from impreproc.images import Image, ImageList
from impreproc.renaming import ImageRenamer

# Define parameters
# Note that in Jupyter Notebooks, the path are relative to the notebooks folder!

# Image path. It can be a single folder containing all images or a folder cointiaing subfolders with the images (recursive option must be set to True)
data_dir = Path("/mnt/labmgf/2023/Sulzano2/img")

# Path to the prior classes file (optional). It is a .csv file with two columns: "name" and "class", without the header. If present, a column "class" will be added to the output Pandas Dataframe
prior_class_file = data_dir / "prior_classes.csv"

# Define extensions of the file to read as a list of strings. 
# It can be a single file extension ["JPG"] or multiples extensions ["jpg", "PNG", "DNG"] 
image_ext = ["jpg"] 

# Read images recursively in subbofolders
recursive = True

# Destination folder
dest_folder = Path("/mnt/labmgf/2023/Sulzano2/img/renamed")

# Base name for the renamed images  
base_name = "IMG"

# Add a progressive ID after the base name
progressive_id = True

# Overlay name on the image
overlay_name = True

# Delete original files
delete_original = False

# Process images in parallel using all the cores available in the machine for speed up the process (it only works when progressive_id is False)
parallel = True

#### Get list of files


In [2]:
# Get image list
files = ImageList(data_dir, image_ext=image_ext, recursive=recursive)

# See first 5 files to check if everything is ok
files.head

/mnt/labmgf/2023/Sulzano2/img/D800/JPG/_DSC1548.JPG
/mnt/labmgf/2023/Sulzano2/img/D800/JPG/_DSC1549.JPG
/mnt/labmgf/2023/Sulzano2/img/D800/JPG/_DSC1550.JPG
/mnt/labmgf/2023/Sulzano2/img/D800/JPG/_DSC1551.JPG
/mnt/labmgf/2023/Sulzano2/img/D800/JPG/_DSC1552.JPG


#### Rename images and saving output to Pandas df

In [3]:
# Create ImageRenamer object
renamer = ImageRenamer(
    image_list=files,
    dest_folder=dest_folder,
    base_name=base_name,
    progressive_ids=progressive_id,
    delete_original=delete_original,
    # parallel=parallel,
    # prior_class_file=prior_class_file,
)

# Rename files and get Pandas Dataframe with old and new names
df = renamer.rename()

# Save Pandas Dataframe as .csv and .parquet file
df.to_csv(dest_folder/"renaming_dict.csv", index=False)
df.to_parquet(dest_folder/"renaming_dict.parquet")

  4%|▍         | 76/1908 [00:38<15:54,  1.92it/s]WARNING:root:Unable to get GPS coordinates from exif from image _DSC1641.JPG.


#### Make previews for Potree

In [ ]:
# Make previews for Potree by overlaying the name on the image
# This method is not fully implemented yet (resizing and normalizing by the focal length is missing)
renamer.make_previews(dest_folder=dest_folder / "previews")